In [7]:
import tensorflow as tf
import numpy as np
import zipfile
import os
import cv2
import pandas as pd
from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
def dataGen(path_train, batch, classes, path_validate=None):
   ''' the dataGen() function creates an ImageDataGenerator of both the the train dataset and validation dataset,
   with it you can specify the type of classification(Binary or Categorical) for the images, and choose its batch size.

   args:
   path_train = string of train dataset path,
   path_validate(optional) = validation dataset path,
   batch = integer of the batch size.
   classes = integer of classes number.

   returns:
   train_generator = ImageDataGenerator of the train dataset,
   validate_generator = ImageDataGenerator of the validation dataset.'''

   # variable to specify the type of classification of our problem.
   c_mode = 'categorical'
   if classes == 2:
         c_mode = 'binary'
   # if statement to check if the user have given a validation path.
   if path_validate == None:
      train_generator = ImageDataGenerator(
          rescale = 1./255,
          shear_range = 0.2,
          zoom_range = 0.3,
          horizontal_flip = True,
          validation_split=0.2
      )

      train_data = train_generator.flow_from_directory(
          directory = path_train,
          target_size = (224, 224),
          batch_size = batch,
          class_mode = c_mode,
          subset = 'training'
      )

      validate_data = train_generator.flow_from_directory(
          directory = path_train,
          target_size = (224, 224),
          batch_size = batch,
          class_mode = c_mode,
          subset = 'validation'
      )

      return train_data, validate_data

   else:
      train_generator = ImageDataGenerator( rescale= 1./255,
                                        shear_range = 0.2,
                                        zoom_range = 0.3,
                                        horizontal_flip = True)

      validate_generator = ImageDataGenerator(rescale= 1./255,
                                          shear_range = 0.2,
                                          zoom_range = 0.3,
                                          horizontal_flip = True)

      train_data = train_generator.flow_from_directory(
            directory = path_train,
            target_size = (224,224),
            batch_size = batch,
            class_mode = c_mode
        )

      validate_data = validate_generator.flow_from_directory(
            directory = path_validate,
            target_size = (224,224),
            batch_size = batch,
            class_mode = c_mode
        )

      return train_data, validate_data

In [10]:
def build(name_of_model, classes):
  ''' the bulid() function creates a keras model based on the VGG-16 architecture, with the build() function you can
  specify the type of classification(Binary or categorical) with the specified integer of classes.

  args:
  name_of_model = string of the desired name of the model,
  num_of_classes = integer of the number of classes for the model.

  returns:
  keras.model object with VGG-16 architecture and the specified number of classes.'''

  # Intialising variables
  (w, h ,c) = 224, 224, 3

  # First block
  input = tf.keras.Input(shape=(w, h, c), name="Input")

  x = layers.Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu", name = "Conv1")(input)
  x = layers.Conv2D(filters = 64, kernel_size = 3, padding = "same", activation = "relu", name = "Conv2")(x)
  x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same", name="Max_pool1")(x)

  # Second block
  x = layers.Conv2D(filters = 128,kernel_size = 3, padding = "same", activation = "relu", name = "Conv3")(x)
  x = layers.Conv2D(filters = 128,kernel_size = 3, padding = "same", activation = "relu", name = "Conv4")(x)
  x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same", name="Max_pool2")(x)

  # Third block
  x = layers.Conv2D(filters = 256, kernel_size = 3, padding = "same", activation="relu", name = "Conv5")(x)
  x = layers.Conv2D(filters = 256, kernel_size = 3, padding = "same", activation="relu", name = "Conv6")(x)
  x = layers.Conv2D(filters = 256, kernel_size = 3, padding = "same", activation="relu", name = "Conv7")(x)
  x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same", name="Max_pool3")(x)

  # Fourth block
  x = layers.Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu", name = "Conv8")(x)
  x = layers.Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu", name = "Conv9")(x)
  x = layers.Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu", name = "Conv10")(x)
  x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same", name="Max_pool4")(x)

  # Fifth block
  x = layers.Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu", name = "Conv11")(x)
  x = layers.Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu", name = "Conv12")(x)
  x = layers.Conv2D(filters = 512, kernel_size = 3, padding = "same", activation = "relu", name = "Conv13")(x)
  x = layers.MaxPooling2D(pool_size=(2,2), strides=(2,2), padding="same", name="Max_pool5")(x)

  # Fully connected layers
  x = layers.Flatten()(x)
  x = layers.Dense(4096, activation="relu", name="Dense1")(x)
  x = layers.Dropout(.35, name="Dropout1")(x)
  x = layers.Dense(4096, activation="relu", name="Dense2")(x)
  x = layers.Dropout(.35, name="Dropout2")(x)
  x = layers.Dense(1000, activation="relu", name="Dense3")(x)
  x = layers.Dropout(.35, name="Dropout3")(x)

  # checking the type of classification.
  if classes == 2:
       output = layers.Dense(1, activation="sigmoid", name="Output")(x)
  else:
       output = layers.Dense(classes, activation="softmax", name="Output")(x)

  model = tf.keras.Model(inputs=input, outputs=output, name=name_of_model)
  return model

In [11]:
def modelCompile(model, classes, learning_rate):
  ''' the modelCompile() function compiles the model based on the number of classes it has.

  args:
  model = keras.model object of the model to be compiled,
  classes = integer of the number of classes the model has.

  returns:
  a compiled keras model.
  '''

  # checking the type of classification done on the model.
  if classes == 2:
    c_mode = tf.keras.losses.BinaryCrossentropy()
  else:
    c_mode = tf.keras.losses.CategoricalCrossentropy()

  # applying decay to the weights and biases.
  lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=learning_rate,
    decay_steps=10000,
    decay_rate=0.0005)

  # compiling the model.
  model.compile(
      loss = c_mode,
      optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
      metrics = 'accuracy'
  )

In [12]:
def modelEvalution(model, test_path, batch, classes):
   ''' the modelEvalution() function takes a model and a test set to evaluate it's loss and accuracy.

   args:
   model = keras.model object used for evalution,
   test_path = string of the path for the test dataset,
   batch = integer of the batch size,
   classes = integer number of classes

   returns:
   prints evalution of the model's performance.
   '''
   # variable to specify the type of classification of our problem
   if classes == 2:
    c_mode = "binary"
   else:
    c_mode = "categorical"

   # generating a test data set with the ImageDataGenerator()
   test_data = ImageDataGenerator(
       rescale=1./255,
       shear_range=0.2,
       zoom_range=0.3,
       horizontal_flip=True

   )

   test_generator = test_data.flow_from_directory(
       directory = test_path,
       target_size = (224,224),
       batch_size = batch,
       class_mode = c_mode

   )

   # print the evaluation of the model
   print(model.evaluate(test_generator))


In [13]:
def main():
  ''' the main() function is the main function that calls the other function.'''

  # variables for when calling the functions.
  NAME = input("now please enter the name of the model: ")
  CLASSES = int(input("please enter the number of classes: "))

  model = build(NAME, CLASSES)
  model.summary()

  train_data_path = input("please enter the path of the train dataset: ")

  # check if the user wants to add validtion set path.
  validation_data_path = None
  validate_y_or_no = input("want to include a validation set, y or n? ")
  if validate_y_or_no == 'y':
      validation_data_path = input("then please enter the path of the validation dataset: ")

  # batch size and learning rate variables.
  BATCH_SIZE = int(input("enter the batch size: "))
  LEARNING_RATE = float(input("please enter the learning rate:"))

  # generating the train and validation data.
  train_gen, valid_gen = dataGen(train_data_path, path_validate=validation_data_path, batch = BATCH_SIZE, classes = CLASSES)

  # compiling the model.
  modelCompile(model, CLASSES, LEARNING_RATE)

  EPOCHS = int(input("please enter the number of epochs: "))

  # adding callbacks for early stopping and saving the best performance of each test.
  EARLYSTOP = callbacks.EarlyStopping(monitor = 'val_accuracy', min_delta=0.005, patience = 10, mode = 'auto')
  CHECKPOINT = callbacks.ModelCheckpoint(f'{NAME}_model.keras', save_best_only=True, monitor='val_accuracy', mode='auto')

  # fitting the generated data.
  model.fit(train_gen,
            batch_size = BATCH_SIZE,
            epochs = EPOCHS,
            validation_data = valid_gen,
            validation_batch_size = BATCH_SIZE,
            callbacks=[EARLYSTOP,CHECKPOINT])

  # asking the user if they want an evaluation.
  evaluate_or_not = input("do want to evaluate the model, y or n? ")
  if evaluate_or_not == 'y':
    test_data_path = input("please provide the path for the testing dataset: ")
    modelEvalution(model, test_data_path, BATCH_SIZE, CLASSES)

  # print statement declaring the program is done.
  print("training done successfully!!")

In [1]:
def modelLoad(path):
  ''' modelLoad() function loads a pre-trained keras model.

  args:
  path = string of the keras model path.

  returns:
  a keras.model object of the pre-trained model.'''
  model = tf.keras.models.load_model(path)

  return model

In [2]:
def singleImageClassification(path, model):
  ''' the singleImageClassification() takes a single image and classifies it.

  args:
  path = string of the image path,
  model = keras.model object used for classification.

  returns:
  tensor of the predicted class of the image.'''

  # try function tries to fetch ih the path of the image is correct.
  try:

  # preprocessing the image before feeding it to the model(e.g. rescale, resizing etc.).
    img = cv2.imread(path)
    img = cv2.resize(img, (224,224))
    img = tf.constant(img)
    img = tf.expand_dims(img, axis=0)/255

    logit = model.predict(img)
    prediction = tf.nn.softmax(logit)

    return np.argmax(prediction, 1)

  except:
    print("incorrect path, please try again.")

In [3]:
def classifyPath(path, model):
  ''' the classifyPath() function takes a path full of images and returns a dictionary of predictions for
   all the images in given path.

   args:
   path = a string of the path containing the images to classify,
   model = keras.model object of the trained model to use for prediction.

   returns:
   a dictionary of al the predictions.'''

  listOfFiles = {}

  # os.listdir to go through the whole file.
  for filename in os.listdir(path):
    if filename.endswith('.jpg'):
     listOfFiles[filename] = singleImageClassification(os.path.join(path, filename), model)

  return listOfFiles

In [ ]:
model = build("VGG-16", 2)
model.summary()

Model: "VGG-16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                                                                 
 Max_pool2 (MaxPooling2D)    (None, 56, 56, 128)       0    

In [ ]:
# Extracting the dataset file
zip_ref = zipfile.ZipFile('/content/catsvsdogs_small.zip')
zip_ref.extractall()
zip_ref.close()

# Testing the Model:

here we'll be conducting several tests on multiple datasets to measure the effictivness of the model.

> Dataset [1] : a small dataset of only 557 pictures belonging to 2 classes, class zero being cats and class one being dogs.

> Dataset [2] : a medium dataset of 5494 images with 12 classes, these classes being different type of pests.

> Dataset [3] : a large dataset with 14034 training images and validation set of 3000 images, with 6 different classes, the classes are different types of sceneries.

# Dataset 1:

the model was too complex for the amount of data we have, even after multiple fine tuning and hyperparameter adjustments, the results were terrible. the model went into overfitting in all tests, best result achieved a validation accuracy of 67.2 % and a loss of 0.66, but when put in evaluation it resulted in an accuracy of 57%.

In [ ]:
main()

now please enter the name of the model: Cats-vs-Dogs
please enter the number of classes: 2
Model: "Cats-vs-Dogs"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                             

45/45 [==============================] - 33s 446ms/step - loss: 0.6931 - accuracy: 0.5123 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 2/100
45/45 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5034

45/45 [==============================] - 12s 260ms/step - loss: 0.6931 - accuracy: 0.5034 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/100
45/45 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5078

45/45 [==============================] - 12s 269ms/step - loss: 0.6931 - accuracy: 0.5078 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 4/100
45/45 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.4989

45/45 [==============================] - 12s 271ms/step - loss: 0.6931 - accuracy: 0.4989 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/100
45/45 [==============================] - ETA: 0s - loss: 0.6931 - accuracy: 0.5034

...

45/45 [==============================] - 12s 272ms/step - loss: 0.6700 - accuracy: 0.6085 - val_loss: 0.6610 - val_accuracy: 0.6364
Epoch 97/100
45/45 [==============================] - ETA: 0s - loss: 0.6694 - accuracy: 0.6264

45/45 [==============================] - 12s 270ms/step - loss: 0.6694 - accuracy: 0.6264 - val_loss: 0.6628 - val_accuracy: 0.6636
Epoch 98/100
45/45 [==============================] - ETA: 0s - loss: 0.6713 - accuracy: 0.5973

45/45 [==============================] - 12s 272ms/step - loss: 0.6713 - accuracy: 0.5973 - val_loss: 0.6619 - val_accuracy: 0.6727
Epoch 99/100
45/45 [==============================] - ETA: 0s - loss: 0.6714 - accuracy: 0.6085

45/45 [==============================] - 12s 272ms/step - loss: 0.6714 - accuracy: 0.6085 - val_loss: 0.6621 - val_accuracy: 0.6273
Epoch 100/100
45/45 [==============================] - ETA: 0s - loss: 0.6704 - accuracy: 0.6063

45/45 [==============================] - 12s 270ms/step - loss: 0.6704 - accuracy: 0.6063 - val_loss: 0.6632 - val_accuracy: 0.6545
do want to evaluate the model, y or n? y
please provide the path for the testing dataset: /content/test
Found 140 images belonging to 2 classes.
14/14 [==============================] - 2s 153ms/step - loss: 0.6790 - accuracy: 0.5786
[0.6790106892585754, 0.5785714387893677]
model saved successfully!!


In [ ]:
main()

now please enter the name of the model: Cats-vs-Dogs3
please enter the number of classes: 2
Model: "Cats-vs-Dogs3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                           

In [ ]:
main()

now please enter the name of the model: Cats-vs-Dogs4
please enter the number of classes: 2
Model: "Cats-vs-Dogs4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                           

# Dataset 2:

here we have a bigger dataset to work on, but even with the larger set the model still performs poorly, with the best performing model having a validation accuracy of 51.5% and and a loss of 1.66, even though the model reached 88.7% accuracy in training, the model is still overfitting, we need a bigger dataset.

In [ ]:
# Extracting the dataset file
zip_ref = zipfile.ZipFile('/content/pesticide.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
main()

now please enter the name of the model: Pesticides
please enter the number of classes: 12
Model: "Pesticides"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                                

In [ ]:
main()

now please enter the name of the model: Pesticides2
please enter the number of classes: 12
Model: "Pesticides2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                              

In [ ]:
main()

now please enter the name of the model: Pesticides3
please enter the number of classes: 12
Model: "Pesticides3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                              

In [ ]:
main()

now please enter the name of the model: Pesticides4
please enter the number of classes: 12
Model: "Pesticides4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                              

# Dataset 3:

this was the most viable dataset for the model, it reached a validation accuracy of 88.3% and a loss of 0.36(the model has an accuracy of 92% on the Imagenet test), clearly the best fitting test so far, altough a higher accuracy is achievable, the model still needs more fine tuning and adjustment and with the lack of computational resources, this dataset needs more testing.

In [25]:
# Extracting the dataset file
zip_ref = zipfile.ZipFile('/content/imagescene.zip')
zip_ref.extractall()
zip_ref.close()

In [ ]:
main()

now please enter the name of the model: Scenery
please enter the number of classes: 6
Model: "Scenery"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                                       

In [17]:
# Extracting the dataset file
zip_ref = zipfile.ZipFile('/content/Scenery_model.zip')
zip_ref.extractall()
zip_ref.close()

In [8]:
model = modelLoad("/content/Scenery_model.keras")

In [9]:
model.summary()

Model: "Scenery"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input (InputLayer)          [(None, 224, 224, 3)]     0         
                                                                 
 Conv1 (Conv2D)              (None, 224, 224, 64)      1792      
                                                                 
 Conv2 (Conv2D)              (None, 224, 224, 64)      36928     
                                                                 
 Max_pool1 (MaxPooling2D)    (None, 112, 112, 64)      0         
                                                                 
 Conv3 (Conv2D)              (None, 112, 112, 128)     73856     
                                                                 
 Conv4 (Conv2D)              (None, 112, 112, 128)     147584    
                                                                 
 Max_pool2 (MaxPooling2D)    (None, 56, 56, 128)       0   

In [10]:
singleImageClassification('/content/download.jpeg', model)

1/1 [==============================] - 1s 926ms/step


array([5])

In [11]:
zip_ref = zipfile.ZipFile('/content/predictimages.zip')
zip_ref.extractall()
zip_ref.close()

In [12]:
predlist = classifyPath('/content/predictimages', model)
predlist

1/1 [==============================] - 1s 571ms/step


{'10012.jpg': array([5]),
 '10045.jpg': array([5]),
 '10034.jpg': array([4]),
 '10005.jpg': array([3]),
 '10017.jpg': array([2]),
 '10043.jpg': array([4]),
 '1003.jpg': array([4]),
 '10004.jpg': array([0]),
 '10021.jpg': array([1]),
 '10040.jpg': array([5]),
 '10038.jpg': array([4]),
 '10013.jpg': array([3])}

# Conclusion:

  VGG-16 is a powerful model architecture, but requires
  a large dataset inorder to perform efficently, the main point to take from this project is it's fine to train such complex models on a dataset, but your better off transferring a pre-trained model for the use case, especially when lacking a good enough set.